In [105]:
!python -m pip install -r ./doc-splitter/requirements.txt
!python -m pip install PyPDF2
!pip install --upgrade google-cloud-documentai

In [112]:
# TODO(developer): Fill these variables with your values before running the sample
!find . -regex '.*test-bd.pdf$'
PROJECT_ID= 'XXXXXXXXXX'
LOCATION = 'us' # Format is 'us' or 'eu'
SPLIT_PROCESSOR_ID = 'XXXXXXXXXX'
IN_PARSE_PROCESSOR_ID = 'XXXXXXXXXX'
FILE_PATH = './doc-splitter/resources/test-bd.pdf'
!export GOOGLE_APPLICATION_CREDENTIALS="./doc-splitter/XXXXXXXXXX.json"

./doc-splitter/resources/test-bd.pdf


In [113]:
# Import necessary Python modules
from google.cloud import documentai_v1beta3 as documentai
from google.oauth2 import service_account
import base64
import json
from PyPDF2 import PdfFileReader as R
import PyPDF2
import time

In [118]:
def process_document(
    project_id: str, location: str, processor_id: str, file_path: str
):

    credentials = service_account.Credentials.from_service_account_file(
    './doc-splitter/raniamoh-playground-docai.json')
    # Instantiates a client
    opts = {"api_endpoint": f"{location}-documentai.googleapis.com"}
    client = documentai.DocumentProcessorServiceClient(client_options=opts, credentials=credentials)

    split_name = f"projects/{project_id}/locations/{location}/processors/{SPLIT_PROCESSOR_ID}/processorVersions/pretrained-next"
    parse_name = f"projects/{project_id}/locations/{location}/processors/{IN_PARSE_PROCESSOR_ID}/processorVersions/pretrained-next"
    with open(file_path, "rb") as image:
        image_content = image.read() 
    # Read the file into memory
    document = {"content": image_content, "mime_type": "application/pdf"}
    # Configure the process request
    request = {"name": split_name, "document": document}
    # Recognizes text entities in the PDF document
    result = client.process_document(request=request)
    document = result.document
    print("Document Split completed.")

    document_pages = document.pages
    print("======")
    # Read the detected page split from the processor
    for i, entity in enumerate(document.entities):
        confidence = entity.confidence
        text_entity = ''              
        pages = [p.page for p in entity.page_anchor.page_refs]
        image2= open(file_path, "rb") 
        pdf = R(image2)   
        for p in pages:
            print(f"====== Parsed document number {p} start ======")
            pdfWriter = PyPDF2.PdfFileWriter()    
            pdfWriter.addPage(pdf.getPage(p))
            newFile = open(f"page.pdf", 'wb')
            pdfWriter.write(newFile)
            newFile.close()
            with open("page.pdf", "rb") as image:
                image_content = image.read()
            document2 = {"content": image_content, "mime_type": "application/pdf"}
            # Configure the process request
            request = {"name": parse_name, "document": document2}
            result = client.process_document(request=request)
            document2 = result.document
            property_index =0
            for i, entity in enumerate(document2.entities):
                if bool (entity.properties):
                    for j, subentity in enumerate(entity.properties):
                        property_index = property_index +1
                        print(f"Reading property number {property_index}")
                        print(f"Type: {subentity.type_}")
                        print(f"Value: {subentity.normalized_value.text}")
                else:
                    property_index = property_index +1
                    print(f"Reading property number {property_index}")
                    print(f"Type: {getattr(entity, 'type_')}")
                    itemValue = getattr(getattr(entity, 'normalized_value'),'text')
                    if (bool(itemValue)):
                        print(f"Value: {itemValue}")
                    else:
                        print(f"Value: {entity.mention_text}")
              print(f"====== Parsed document number {p} End ======")




In [119]:
process_document(PROJECT_ID, LOCATION, PROCESSOR_ID, FILE_PATH)

Document Split completed.
Reading property number 1
Type: currency
Value: GBP
Reading property number 2
Type: line_item/description
Value: Tea - Tali - Union H
Reading property number 3
Type: line_item/amount
Value: 0.61
Reading property number 4
Type: line_item/description
Value: Americano - Tall- U
Reading property number 5
Type: line_item/amount
Value: 1.32
Reading property number 6
Type: line_item/description
Value: Americano - Tall - U
Reading property number 7
Type: line_item/amount
Value: 1.32
Reading property number 8
Type: purchase_time
Value: 10:19 : Debbie 
Reading property number 9
Type: receipt_date
Value: 2019-08-22
Reading property number 10
Type: supplier_address
Value: Longwood Close
Westwood Business Park
Coventry
CV4 8)
Reading property number 11
Type: supplier_city
Value: Coventry
Reading property number 12
Type: supplier_name
Value: Westwood Business Park
Reading property number 13
Type: total_amount
Value: 3.25
Reading property number 14
Type: total_tax_amount
Val